In [1]:
import os
import pandas as pd
import numpy as np
import json
from glob import glob
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [24]:
# VinDR-CXR
df_vindr = pd.read_csv('/home/than/DeepLearning/cxr_concept/CheXzero/benchmark/results/simplified_auc_comparison_padchestt25_indianat25_vindrcxrt10/vindrcxr_results.csv')
df_vindr.head()
print(len(df_vindr))

,dataset,label,n_positive,n_total,SFR-Mistral_auc,SFR-Mistral_ci_lower,SFR-Mistral_ci_upper,CheXzero_auc,CheXzero_ci_lower,CheXzero_ci_upper,...,Qwen3_8B_auc,Qwen3_8B_ci_lower,Qwen3_8B_ci_upper,SFR-Mistral_vs_Qwen3_8B_p_value,SFR-Mistral_vs_Qwen3_8B_diff,OpenAI_Small_auc,OpenAI_Small_ci_lower,OpenAI_Small_ci_upper,SFR-Mistral_vs_OpenAI_Small_p_value,SFR-Mistral_vs_OpenAI_Small_diff
0,vindrcxr,Aortic enlargement,220,3000,0.709072,0.672759,0.744945,0.758169,0.726431,0.788347,...,0.689811,0.657301,0.721354,0.418,0.019261,0.692606,0.656962,0.727165,0.094,0.016466
1,vindrcxr,Atelectasis,86,3000,0.785012,0.735317,0.833880,0.744658,0.695972,0.789085,...,0.739484,0.704958,0.773259,0.355,0.045528,0.745966,0.707415,0.782421,0.480,0.039046
2,vindrcxr,Calcification,194,3000,0.616881,0.573084,0.657756,0.760297,0.728127,0.790612,...,0.531498,0.491595,0.572785,0.008,0.085383,0.634945,0.600961,0.668048,0.627,-0.018065
3,vindrcxr,Cardiomegaly,309,3000,0.782209,0.753100,0.810496,0.936161,0.923614,0.947754,...,0.757241,0.730655,0.782137,0.137,0.024968,0.636592,0.609670,0.662087,0.000,0.145617
4,vindrcxr,Consolidation,96,3000,0.876682,0.854773,0.895934,0.905412,0.868543,0.938556,...,0.802205,0.747093,0.850328,0.078,0.074477,0.825599,0.795915,0.852382,0.026,0.051082


21


In [22]:
# compare with CheXzero
cols = [
        'dataset','label','n_positive',
        'SFR-Mistral_auc','SFR-Mistral_ci_lower','SFR-Mistral_ci_upper',
        'CheXzero_auc','CheXzero_ci_lower','CheXzero_ci_upper',
        'SFR-Mistral_vs_CheXzero_p_value',
        'SFR-Mistral_vs_CheXzero_diff'
        ]
# SFR-mistral vs. CheXzero
df = df_vindr[cols]
# compute average AUC
df['SFR-Mistral_auc'].mean(), df['CheXzero_auc'].mean()
# significant difference
df = df[df['SFR-Mistral_vs_CheXzero_p_value'] < 0.01].sort_values(by=['SFR-Mistral_vs_CheXzero_diff'], ascending=False)
print(len(df[df['SFR-Mistral_vs_CheXzero_diff'] > 0])) 
print(len(df[df['SFR-Mistral_vs_CheXzero_diff'] < 0]))
df

(0.781840619047619, 0.749660619047619)

4
3


,dataset,label,n_positive,SFR-Mistral_auc,SFR-Mistral_ci_lower,SFR-Mistral_ci_upper,CheXzero_auc,CheXzero_ci_lower,CheXzero_ci_upper,SFR-Mistral_vs_CheXzero_p_value,SFR-Mistral_vs_CheXzero_diff
10,vindrcxr,No finding,2051,0.693242,0.671947,0.713789,0.139005,0.124607,0.153971,0.000,0.554238
11,vindrcxr,Nodule/Mass,176,0.792947,0.753748,0.829352,0.661605,0.622084,0.700420,0.000,0.131342
18,vindrcxr,Pulmonary fibrosis,217,0.767617,0.734528,0.800231,0.667708,0.630831,0.706219,0.000,0.099909
8,vindrcxr,Lung tumor,80,0.884212,0.846815,0.918275,0.809695,0.757030,0.859799,0.001,0.074518
0,vindrcxr,Aortic enlargement,220,0.709072,0.672759,0.744945,0.758169,0.726431,0.788347,0.007,-0.049097
2,vindrcxr,Calcification,194,0.616881,0.573084,0.657756,0.760297,0.728127,0.790612,0.000,-0.143416
3,vindrcxr,Cardiomegaly,309,0.782209,0.753100,0.810496,0.936161,0.923614,0.947754,0.000,-0.153951


In [21]:
# compare with BiomedCLIP
cols = [
        'dataset','label','n_positive',
        'SFR-Mistral_auc','SFR-Mistral_ci_lower','SFR-Mistral_ci_upper',
        'BiomedCLIP_auc','BiomedCLIP_ci_lower','BiomedCLIP_ci_upper',
        'SFR-Mistral_vs_BiomedCLIP_p_value',
        'SFR-Mistral_vs_BiomedCLIP_diff'
        ]
# SFR-mistral vs. CheXzero
df = df_vindr[cols]
# compute average AUC
df['SFR-Mistral_auc'].mean(), df['BiomedCLIP_auc'].mean()
# significant difference
df = df[df['SFR-Mistral_vs_BiomedCLIP_p_value'] < 0.01].sort_values(by=['SFR-Mistral_vs_BiomedCLIP_diff'], ascending=False)
print(len(df[df['SFR-Mistral_vs_BiomedCLIP_diff'] > 0])) 
print(len(df[df['SFR-Mistral_vs_BiomedCLIP_diff'] < 0]))
df

(0.781840619047619, 0.6405534285714286)

9
0


,dataset,label,n_positive,SFR-Mistral_auc,SFR-Mistral_ci_lower,SFR-Mistral_ci_upper,BiomedCLIP_auc,BiomedCLIP_ci_lower,BiomedCLIP_ci_upper,SFR-Mistral_vs_BiomedCLIP_p_value,SFR-Mistral_vs_BiomedCLIP_diff
10,vindrcxr,No finding,2051,0.693242,0.671947,0.713789,0.368716,0.345010,0.389832,0.000,0.324526
6,vindrcxr,Infiltration,58,0.724755,0.651537,0.791245,0.451156,0.372709,0.534043,0.000,0.273599
16,vindrcxr,Pneumonia,246,0.937425,0.922027,0.950422,0.674650,0.636268,0.712776,0.000,0.262775
4,vindrcxr,Consolidation,96,0.876682,0.854773,0.895934,0.614654,0.552232,0.678839,0.000,0.262028
11,vindrcxr,Nodule/Mass,176,0.792947,0.753748,0.829352,0.536141,0.489570,0.582876,0.000,0.256806
13,vindrcxr,Other lesion,94,0.762761,0.719733,0.807654,0.559767,0.499361,0.623565,0.000,0.202994
12,vindrcxr,Other disease,657,0.636212,0.616418,0.656100,0.454916,0.430162,0.481030,0.000,0.181296
18,vindrcxr,Pulmonary fibrosis,217,0.767617,0.734528,0.800231,0.656524,0.616888,0.695527,0.002,0.111093
5,vindrcxr,ILD,221,0.816141,0.789172,0.840342,0.711104,0.674977,0.748698,0.005,0.105037


In [23]:
# compare with OpenAI CLIP
cols = [
        'dataset','label','n_positive',
        'SFR-Mistral_auc','SFR-Mistral_ci_lower','SFR-Mistral_ci_upper',
        'OpenAI_CLIP_auc','OpenAI_CLIP_ci_lower','OpenAI_CLIP_ci_upper',
        'SFR-Mistral_vs_OpenAI_CLIP_p_value',
        'SFR-Mistral_vs_OpenAI_CLIP_diff'
        ]
# SFR-mistral vs. CheXzero
df = df_vindr[cols]
# compute average AUC
df['SFR-Mistral_auc'].mean(), df['OpenAI_CLIP_auc'].mean()
# significant difference
df = df[df['SFR-Mistral_vs_OpenAI_CLIP_p_value'] < 0.01].sort_values(by=['SFR-Mistral_vs_OpenAI_CLIP_diff'], ascending=False)
print(len(df[df['SFR-Mistral_vs_OpenAI_CLIP_diff'] > 0])) 
print(len(df[df['SFR-Mistral_vs_OpenAI_CLIP_diff'] < 0]))
df

(0.781840619047619, 0.5265383809523809)

20
0


,dataset,label,n_positive,SFR-Mistral_auc,SFR-Mistral_ci_lower,SFR-Mistral_ci_upper,OpenAI_CLIP_auc,OpenAI_CLIP_ci_lower,OpenAI_CLIP_ci_upper,SFR-Mistral_vs_OpenAI_CLIP_p_value,SFR-Mistral_vs_OpenAI_CLIP_diff
4,vindrcxr,Consolidation,96,0.876682,0.854773,0.895934,0.421719,0.365235,0.478384,0.000,0.454963
17,vindrcxr,Pneumothorax,18,0.878829,0.837036,0.918181,0.450063,0.341667,0.570831,0.000,0.428766
16,vindrcxr,Pneumonia,246,0.937425,0.922027,0.950422,0.544844,0.507379,0.583427,0.000,0.392581
9,vindrcxr,Mediastinal shift,20,0.877677,0.825396,0.922970,0.485280,0.353924,0.606417,0.000,0.392397
14,vindrcxr,Pleural effusion,111,0.904063,0.881976,0.923404,0.529099,0.469421,0.587346,0.000,0.374964
8,vindrcxr,Lung tumor,80,0.884212,0.846815,0.918275,0.540571,0.480083,0.597968,0.000,0.343641
19,vindrcxr,Rib fracture,11,0.776058,0.658915,0.872757,0.491053,0.324644,0.642571,0.009,0.285006
7,vindrcxr,Lung Opacity,84,0.808731,0.767729,0.845341,0.557876,0.488340,0.625130,0.000,0.250855
2,vindrcxr,Calcification,194,0.616881,0.573084,0.657756,0.381824,0.336399,0.424376,0.000,0.235057
1,vindrcxr,Atelectasis,86,0.785012,0.735317,0.833880,0.554001,0.495546,0.614609,0.000,0.231011
